## Great Recession vs Covid-19
To compare the FTSE 100 prices of 2008 crash and 2020 crash

In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "notebook"

In [2]:
BASE_URL = 'https://docs.google.com/spreadsheets/d/10cihRP6XxJOSrWsASfW5C7VeHufD32TS0JKCXLoWhL8/export?format=csv&{}'

UKX_sheet = BASE_URL.format('gid=134530274')
HUKX_sheet = BASE_URL.format('gid=1614447400')
ISF_sheet = BASE_URL.format('gid=1374752918')


In [3]:
df = pd.read_csv(UKX_sheet)

# Rename columns
df = df.rename(columns={'Unnamed: 1': 'Open'})

# Convert to datetime
df['Date'] = pd.to_datetime(df['Date'].str[:10], format="%d/%m/%Y")

# Get datetime related fields
df['month'] = df['Date'].dt.month_name()
df['year'] = df['Date'].dt.year
df['day'] = df['Date'].dt.day
df['day_name'] = df['Date'].dt.day_name()
df['weekofyear'] = df['Date'].dt.weekofyear
df['is_quarter_end'] = df['Date'].dt.is_quarter_end
df['is_quarter_start'] = df['Date'].dt.is_quarter_start
df['quarter'] = df['Date'].dt.quarter
df['avg'] = round((df['High'] + df['Low']) / 2, 2) # $\frac{high + low}{2}$

df = df.set_index('Date', drop=True)
df.head()

,Open,High,Low,Close,Volume,month,year,day,day_name,weekofyear,is_quarter_end,is_quarter_start,quarter,avg
Date,,,,,,,,,,,,,,
1990-01-02,2442.4,2443.3,2425.5,2434.1,0,January,1990,2,Tuesday,1,False,False,1,2434.40
1990-01-03,2451.3,2466.2,2445.8,2463.7,0,January,1990,3,Wednesday,1,False,False,1,2456.00
1990-01-04,2469.6,2479.4,2451.6,2451.6,0,January,1990,4,Thursday,1,False,False,1,2465.50
1990-01-05,2442.0,2448.7,2436.6,2444.5,0,January,1990,5,Friday,1,False,False,1,2442.65
1990-01-08,2445.1,2445.6,2423.8,2431.3,0,January,1990,8,Monday,2,False,False,1,2434.70


In [4]:
recession_df = df['2007-06-15': '2010-01-01'].copy()
covid_df = df['2020-01-17':].copy()

recession_highest_close = recession_df.iloc[0]['Close']
recession_df['drop_percent'] = round(((recession_df['Close'] - recession_highest_close)/recession_highest_close) * 100, 2)


covid_highest_close = covid_df.iloc[0]['Close']
covid_df['drop_percent'] = round(((covid_df['Close'] - covid_highest_close)/covid_highest_close) * 100, 2)

recession_days = list(range(recession_df.shape[0] + 1))



In [27]:
recession_df = recession_df[['Close', 'drop_percent']]
recession_df.index = range(1, recession_df.shape[0] + 1)

covid_df = covid_df[['Close', 'drop_percent']]
covid_df.index = range(1, covid_df.shape[0] + 1)

result_df = pd.merge(recession_df, covid_df, left_index=True, right_index=True, how='left')
result_df['day'] = list(range(1, test.shape[0] + 1))

result_df.to_csv('../data/output/ftse100_2008_vs_covid_period.csv', index=False)

In [26]:
fig = go.Figure()



fig.add_trace(go.Scatter(
                x=result_df['day'],
                y=result_df['drop_percent_x'],
                name="Recession 2008",
                line_color='grey',
                opacity=0.8)
             )


fig.add_trace(go.Scatter(
                x=result_df['day'],
                y=result_df['drop_percent_y'],
                name="covid",
                line_color='red',
                opacity=0.8)
             )


fig.update_layout(
    title_text="FTSE 100 Percentage Drop by Day", 
    xaxis_range=[1, recession_days[-1]])


fig.show()